In [1]:
using MLJ, DataFrames, CSV
import Logging

Logging.disable_logging(Logging.LogLevel(999));  # Disable info logs, but keep warnings

In [2]:
df = CSV.read("./data/401ksubs.csv", DataFrame)
#coerce!(df, Count => Continuous)
first(df, 5)

,net_tfa,age,inc,fsize,educ,db,marr,twoearn,e401,p401,pira
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,47,6765,2,8,0,0,0,0,0,0
2,1015,36,28452,1,16,0,0,0,0,0,0
3,-2000,37,3300,6,12,1,0,0,0,0,0
4,15000,58,52590,2,16,0,1,1,0,0,0
5,0,32,21804,1,11,0,0,0,0,0,0


In [3]:
#describe(df)

# Part A

Transformations

In [4]:
# Get correct types
df[!, 1:5] = coerce(df[!, 1:5], Count => Continuous)
df[!, 6:end] = coerce(df[!, 6:end], Count => Multiclass)
schema(df) |> pretty

┌─────────┬───────────────┬─────────────────────────────────┐
│ names   │ scitypes      │ types                           │
│ Symbol  │ Type          │ Type                            │
│ Unknown │ Unknown       │ Unknown                         │
├─────────┼───────────────┼─────────────────────────────────┤
│ net_tfa │ Continuous    │ Float64                         │
│ age     │ Continuous    │ Float64                         │
│ inc     │ Continuous    │ Float64                         │
│ fsize   │ Continuous    │ Float64                         │
│ educ    │ Continuous    │ Float64                         │
│ db      │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ marr    │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ twoearn │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ e401    │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ p401    │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ pira    │ Multiclass{2} │ CategoricalValue{Int64, UInt32} │
│ hown  

In [5]:
frac = 6000 / size(df)[1]  # Fraction of data that equals 6000 obs
df, df_test = partition(df, frac, rng=42)

X_cols = split("age,inc,fsize,educ,db,marr,twoearn,pira,hown", ",")
X, y = df[!, X_cols], df[!, :net_tfa]
println("Size of X = ",size(X))
first(X, 4)

Size of X = (6000, 9)


,age,inc,fsize,educ,db,marr,twoearn,pira,hown
,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…,Cat…
1,43.0,47094.0,4.0,14.0,0,1,0,0,1
2,31.0,30315.0,4.0,13.0,0,1,0,0,1
3,27.0,38640.0,2.0,16.0,1,1,1,0,0
4,62.0,13317.0,1.0,11.0,0,0,0,0,1


### 1. Estimate $E[net\_tfa]$

In [6]:
mean(y)

18367.1

### 2. Using linear regression estimate $E[net\_tfa|X = \{age, inc, fsize, educ, db, marr, twoearn, pira, hown\}]$

In [7]:
LinearRegressor = (@load LinearRegressor pkg=MLJLinearModels verbosity=0)()
lm_pipe = ContinuousEncoder(drop_last=true) |> LinearRegressor
lm = machine(lm_pipe, X, y) |> fit!;

In [8]:
# Display model results
fp = fitted_params(lm).linear_regressor
coefs = fp.coefs
intercept = fp.intercept
println("Intercept: $(round(intercept, sigdigits=3))")
for (name, val) in coefs
    println("$(rpad(name, 8)):  $(round(val, sigdigits=3))")
end

Intercept: -30800.0
age     :  657.0
inc     :  0.991
fsize   :  -855.0
educ    :  -364.0
db__0   :  3540.0
marr__0 :  624.0
twoearn__0:  18700.0
pira__0 :  -27700.0
hown__0 :  -1910.0


### 3. Lasso

In [10]:
LassoRegressor = @load LassoRegressor pkg=MLJLinearModels

import MLJLinearModels ✔


MLJLinearModels.LassoRegressor

In [54]:
names(temp)

22-element Vector{String}:
 "age"
 "inc"
 "fsize"
 "educ"
 "db__0"
 "db__1"
 "marr__0"
 "marr__1"
 "twoearn__0"
 "twoearn__1"
 ⋮
 "hown__1"
 "age_sq"
 "inc_sq"
 "fsize_sq"
 "educ_sq"
 "age_function"
 "inc_function"
 "fsize_function"
 "educ_function"

In [11]:
""" Nonlinear transformations of data - take interactions and add higher-order terms """
function nl_transform(df)
    # Take interaction terms of all variables
    cols = names(df)
    cat_cols = [:db, :marr, :twoearn, :pira, :hown]

    # Add a second and third exponent of all numerical columns
    num_cols = [:age, :inc, :fsize, :educ]
    sq(x) = x.^2
    cub(x) = x.^3
    for p in [sq, cub]
        transform!(df, num_cols .=> p)
    end

    # Take interaction terms between everything
    df[!, cat_cols] = coerce(df[!, cat_cols], Multiclass => Continuous)  # Transform Categorical to Continuous
    for col1 in cols
        for col2 in cols
            c1 = split(col1, "_")[1]
            c2 = split(col2, "_")[1]
            if c1 != c2
                df[!, col1*"_"*col2] = df[!, col1] .* df[!, col2]
            end
        end
    end
    
    df[!, cat_cols] = coerce(df[!, cat_cols], Continuous => Multiclass)  # Convert back to Categorical type
    
    return df
end
   
X_flex = copy(X)
nl_transform(X_flex)

first(X_flex, 3)

,age,inc,fsize,educ,db,marr,twoearn,pira,hown,age_inc
,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…,Cat…,Float64
1,43.0,47094.0,4.0,14.0,1.0,2.0,1.0,1.0,2.0,2.02504e6
2,31.0,30315.0,4.0,13.0,1.0,2.0,1.0,1.0,2.0,939765.0
3,27.0,38640.0,2.0,16.0,2.0,2.0,2.0,1.0,1.0,1.04328e6


In [12]:
# Standardize numerical variables then onehot encode categorical variables
transformer = Standardizer() |> OneHotEncoder()
transformer = fit!(machine(transformer, X_flex))
X_flex = MLJ.transform(transformer, X_flex);
first(X_flex, 3)

,age,inc,fsize,educ,db__1.0,db__2.0,marr__1.0,marr__2.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.18438,0.408248,0.73133,0.280414,1.0,0.0,0.0,1.0
2,-0.973723,-0.267984,0.73133,-0.0701765,1.0,0.0,0.0,1.0
3,-1.35976,0.0675325,-0.557925,0.981595,0.0,1.0,0.0,1.0


In [13]:
r  = range(LassoRegressor(), :(lambda), lower=50, upper=1000)
tm = TunedModel(model=LassoRegressor(), ranges=r, tuning=Grid(resolution=50),
                resampling=CV(nfolds=5, rng=42), measure=rms)

lasso_tm = machine(tm, X_flex, y) |> fit!
best_mdl = fitted_params(lasso_tm).best_model

# Print results
coefs, intercept = fitted_params(lasso_tm).best_fitted_params

# Get non-zero coefs
coef_vals = [c[2] for c in coefs]

println("Opimtal penalty: ", round(best_mdl.lambda, sigdigits=4))
println("Pct. variables set to zero: ", sum(coef_vals .≈ 0) / length(coefs) )
@show intercept
coefs[.~(coef_vals .≈ 0)]

Evaluating over 50 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 50 metamodels:   2%[>                        ]  ETA: 0:04:31

Evaluating over 50 metamodels:   4%[=>                       ]  ETA: 0:02:54

Evaluating over 50 metamodels:   6%[=>                       ]  ETA: 0:02:12

Evaluating over 50 metamodels:   8%[==>                      ]  ETA: 0:01:51

...

Evaluating over 50 metamodels:  96%[========================>]  ETA: 0:00:04

Evaluating over 50 metamodels:  98%[========================>]  ETA: 0:00:02

Evaluating over 50 metamodels: 100%[=========================] Time: 0:01:27


Opimtal penalty: 166.3
Pct. variables set to zero: 0.6444444444444445
intercept = 18367.0775975277


32-element Vector{Pair{Symbol, Float64}}:
          :age => -4715.13341760039
      :age_inc => 13465.489628538928
    :age_fsize => -582.0123080401257
       :age_db => -319.22073355164315
     :age_marr => -2670.7094108157703
     :age_pira => 7621.417379531911
      :inc_age => 13465.489628538928
     :inc_educ => -646.141215299786
       :inc_db => -93.04040127521098
  :inc_twoearn => -21256.43211814171
               ⋮
 :twoearn_educ => 1535.0736060177737
   :twoearn_db => 1251.6870439797135
 :twoearn_marr => 4792.9502176273645
     :pira_age => 7621.417379531911
     :pira_inc => 3276.515562947317
   :pira_fsize => -486.16618832854385
      :pira_db => -1261.0432773911764
     :hown_inc => 5082.946143835215
 :inc_function => 21615.404196535506

### Ridge

In [14]:
RidgeRegressor = @load RidgeRegressor pkg=MLJLinearModels;

import MLJLinearModels ✔


In [15]:
r  = range(RidgeRegressor(), :(lambda), lower=0.0001, upper=100_000, scale=:log10)
tm = TunedModel(model=RidgeRegressor(), ranges=r, tuning=Grid(resolution=50),
                resampling=CV(nfolds=5, rng=42), measure=rms)

ridge_tm = machine(tm, X_flex, y) |> fit!
best_mdl = fitted_params(ridge_tm).best_model

# Print results
println("Opimtal penalty: ", round(best_mdl.lambda, sigdigits=4))
coefs, intercept = fitted_params(ridge_tm).best_fitted_params
@show intercept
coefs

Evaluating over 50 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 50 metamodels:   2%[>                        ]  ETA: 0:01:08

Evaluating over 50 metamodels:   4%[=>                       ]  ETA: 0:00:36

Evaluating over 50 metamodels:   6%[=>                       ]  ETA: 0:00:25

Evaluating over 50 metamodels:   8%[==>                      ]  ETA: 0:00:19

...

Evaluating over 50 metamodels:  96%[========================>]  ETA: 0:00:00

Evaluating over 50 metamodels:  98%[========================>]  ETA: 0:00:00

Evaluating over 50 metamodels: 100%[=========================] Time: 0:00:04


Opimtal penalty: 0.016
intercept = 20181.181137777075


90-element Vector{Pair{Symbol, Float64}}:
                   :age => -5773.360700886404
                   :inc => -4050.4141027977093
                 :fsize => 4825.476461388463
                  :educ => -3351.9749235394675
      Symbol("db__1.0") => -3433.4536972976907
      Symbol("db__2.0") => 3433.4536972978026
    Symbol("marr__1.0") => -691.5340694592377
    Symbol("marr__2.0") => 691.5340694587849
 Symbol("twoearn__1.0") => -2844.896419821987
 Symbol("twoearn__2.0") => 2844.8964198217304
                        ⋮
             :hown_educ => 1577.9442814976662
               :hown_db => 131.62317012586033
             :hown_marr => -630.3233015039707
          :hown_twoearn => 74.2557458460717
             :hown_pira => 1089.3600651248287
          :age_function => 2528.2035189024764
          :inc_function => 20510.33124092094
        :fsize_function => -428.08189568616007
         :educ_function => 937.7237018396268

### Decision tree

In [16]:
X = machine(ContinuousEncoder(), X) |> fit! |> MLJ.transform;  # Onehot-encode categorical variables

In [17]:
DecisionTreeRegressor = @load DecisionTreeRegressor pkg=DecisionTree

import MLJDecisionTreeInterface ✔


MLJDecisionTreeInterface.DecisionTreeRegressor

In [18]:
r1 = range(DecisionTreeRegressor(), :max_depth, lower=1, upper=5)
r2 = range(DecisionTreeRegressor(), :min_samples_leaf, lower=5, upper=100,)

tm = TunedModel(model=DecisionTreeRegressor(), ranges=[r1, r2], tuning=Grid(resolution=50),
                resampling=CV(nfolds=5, rng=42), measure=rms)

tree = machine(tm, X, y) |> fit!

fp = fitted_params(tree).best_model

Evaluating over 250 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 250 metamodels:   0%[>                        ]  ETA: 0:07:57

Evaluating over 250 metamodels:   1%[>                        ]  ETA: 0:04:31

Evaluating over 250 metamodels:   1%[>                        ]  ETA: 0:03:02

Evaluating over 250 metamodels:   2%[>                        ]  ETA: 0:02:17

...

Evaluating over 250 metamodels:  99%[========================>]  ETA: 0:00:00

Evaluating over 250 metamodels:  99%[========================>]  ETA: 0:00:00

Evaluating over 250 metamodels: 100%[========================>]  ETA: 0:00:00

Evaluating over 250 metamodels: 100%[=========================] Time: 0:00:06


DecisionTreeRegressor(
  max_depth = 3, 
  min_samples_leaf = 7, 
  min_samples_split = 2, 
  min_purity_increase = 0.0, 
  n_subfeatures = 0, 
  post_prune = false, 
  merge_purity_threshold = 1.0, 
  rng = Random._GLOBAL_RNG())

In [19]:
import DecisionTree
DecisionTree.print_tree(tree.fitresult.fitresult, 5)

Feature 2 < 162800.0 ?
├─ Feature 12 < 0.5 ?
    ├─ Feature 2 < 65620.0 ?
        ├─ 3674.0157498824638 : 0/4254
        └─ 37326.37455830388 : 0/283
    └─ Feature 2 < 63690.0 ?
        ├─ 38363.41998060136 : 0/1031
        └─ 91199.4893111639 : 0/421
└─ 551118.3636363636 : 0/11


### Random Forest

In [20]:
forest = EnsembleModel(model=DecisionTreeRegressor())

DeterministicEnsembleModel(
  model = DecisionTreeRegressor(
        max_depth = -1, 
        min_samples_leaf = 5, 
        min_samples_split = 2, 
        min_purity_increase = 0.0, 
        n_subfeatures = 0, 
        post_prune = false, 
        merge_purity_threshold = 1.0, 
        rng = Random._GLOBAL_RNG()), 
  atomic_weights = Float64[], 
  bagging_fraction = 0.8, 
  rng = Random._GLOBAL_RNG(), 
  n = 100, 
  acceleration = CPU1{Nothing}(nothing), 
  out_of_bag_measure = Any[])

In [21]:
r1 = range(forest, :(model.n_subfeatures), lower=1, upper=5)
r2 = range(forest, :bagging_fraction, lower=0.4, upper=1.0)
tm = TunedModel(model=forest, tuning=Grid(resolution=12),
                resampling=CV(nfolds=6), ranges=[r1, r2],
                measure=rms)

forest_tm = machine(tm, X, y) |> fit!

best_mdl = fitted_params(forest_tm).best_model
@show n_bootstraps = best_mdl.n
@show n_subfeatures = best_mdl.model.n_subfeatures
@show bagging_fraction = best_mdl.bagging_fraction;

Evaluating over 60 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 60 metamodels:   2%[>                        ]  ETA: 0:02:43

Evaluating over 60 metamodels:   3%[>                        ]  ETA: 0:02:03

Evaluating over 60 metamodels:   5%[=>                       ]  ETA: 0:01:52

Evaluating over 60 metamodels:   7%[=>                       ]  ETA: 0:01:31

...

Evaluating over 60 metamodels:  95%[=======================> ]  ETA: 0:00:04

Evaluating over 60 metamodels:  97%[========================>]  ETA: 0:00:03

Evaluating over 60 metamodels:  98%[========================>]  ETA: 0:00:01

Evaluating over 60 metamodels: 100%[=========================] Time: 0:01:28


n_bootstraps = best_mdl.n = 100
n_subfeatures = best_mdl.model.n_subfeatures = 5
bagging_fraction = best_mdl.bagging_fraction = 0.6727272727272727


using ShapML

function predict_function(model, data)
  data_pred = DataFrame(y_pred = predict(model, data))
  return data_pred
end

explain = copy(X)
reference = copy(X)

@time data_shap = ShapML.shap(explain = explain,
                        model = forest_tm,
                        predict_function = predict_function,
                        sample_size = 60,
                        seed = 42
                        );
first(data_shap, 2)

using Gadfly

function plot_shap(data_shap)
    data_plot = combine(groupby(data_shap, :feature_name), :shap_effect => (x -> mean(abs.(x)) ) => :mean_effect )
    data_plot = sort(data_plot, order(:mean_effect, rev = true))

    baseline = round(data_shap.intercept[1], digits = 1)

    p = plot(data_plot, y = :feature_name, x = :mean_effect, Coord.cartesian(yflip = true),
             Scale.y_discrete, Geom.bar(position = :dodge, orientation = :horizontal),
             Theme(bar_spacing = 1mm),
             Guide.xlabel("|Shapley effect| (baseline = $baseline)"), Guide.ylabel(nothing),
             Guide.title("Feature Importance - Mean Absolute Shapley Value"))
end


p = plot_shap(data_shap)

### Boosted trees

In [22]:
XGBR = @load XGBoostRegressor;

import MLJXGBoostInterface ✔


In [23]:
r1 = range(XGBR(), :num_round, lower=1, upper=20)
r2 = range(XGBR(), :max_depth, lower=1, upper=5)

tm = TunedModel(model=XGBR(), tuning=Grid(resolution=50),
                resampling=CV(nfolds=5 ,rng=42), ranges=[r1, r2],
                measure=rms)

xgb = machine(tm, X, y) |> fit!
best_mdl = fitted_params(xgb).best_model
@show num_round = best_mdl.num_round
@show max_depth = best_mdl.max_depth;

Evaluating over 100 metamodels:   0%[>                        ]  ETA: N/A

Evaluating over 100 metamodels:   1%[>                        ]  ETA: 0:02:45

Evaluating over 100 metamodels:   2%[>                        ]  ETA: 0:01:24

Evaluating over 100 metamodels:   3%[>                        ]  ETA: 0:00:57

Evaluating over 100 metamodels:   4%[=>                       ]  ETA: 0:00:43

...

Evaluating over 100 metamodels:  97%[========================>]  ETA: 0:00:00

Evaluating over 100 metamodels:  98%[========================>]  ETA: 0:00:00

Evaluating over 100 metamodels:  99%[========================>]  ETA: 0:00:00

Evaluating over 100 metamodels: 100%[=========================] Time: 0:00:09


num_round = best_mdl.num_round = 17
max_depth = best_mdl.max_depth = 2


@time data_shap = ShapML.shap(explain = explain,
                        model = xgb,
                        predict_function = predict_function,
                        sample_size = 60,
                        seed = 42
                        );

plot_shap(data_shap)

### Neural net

In [24]:
import MLJFlux
import Flux
NeuralNetworkRegressor = @load NeuralNetworkRegressor;

import MLJFlux ✔


In [25]:
mutable struct MyNetworkBuilder <: MLJFlux.Builder
    n1::Int #Number of cells in the first hidden layer
    n2::Int #Number of cells in the second hidden layer
    dropout::Float64
    σ
end

MyNetworkBuilder(; n1=16, n2=8, dropout=0.5, σ=Flux.relu) = MyNetworkBuilder(n1, n2, dropout, σ)

function MLJFlux.build(model::MyNetworkBuilder, rng, n_in, n_out)
    n1, n2 = model.n1, model.n2
    dropout, σ = model.dropout, model.σ
    
    init = Flux.glorot_uniform(rng)
    layer1 = Flux.Dense(n_in, n1, σ, init=init)
    layer1_drop = Flux.Dropout(dropout)
    layer2 = Flux.Dense(n1, n2, σ, init=init)
    layer2_drop = Flux.Dropout(dropout)
    layer3 = Flux.Dense(model.n2, n_out, init=init)
    return Flux.Chain(layer1, layer1_drop, layer2, layer2_drop, layer3)
end

nnregressor = NeuralNetworkRegressor(builder=MyNetworkBuilder(), epochs=10)

NeuralNetworkRegressor(
  builder = MyNetworkBuilder(
        n1 = 16, 
        n2 = 8, 
        dropout = 0.5, 
        σ = NNlib.relu), 
  optimiser = Flux.Optimise.Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), 
  loss = Flux.Losses.mse, 
  epochs = 10, 
  batch_size = 1, 
  lambda = 0.0, 
  alpha = 0.0, 
  rng = Random._GLOBAL_RNG(), 
  optimiser_changes_trigger_retraining = false, 
  acceleration = CPU1{Nothing}(nothing))

In [26]:
nn = machine(nnregressor, X, y) |> fit!

Optimising neural net:  18%[====>                    ]  ETA: 0:00:40

Optimising neural net:  27%[======>                  ]  ETA: 0:00:41

Optimising neural net:  36%[=========>               ]  ETA: 0:00:38

Optimising neural net:  45%[===========>             ]  ETA: 0:00:34

Optimising neural net:  55%[=============>           ]  ETA: 0:00:29

...

Optimising neural net:  73%[==================>      ]  ETA: 0:00:18

Optimising neural net:  82%[====================>    ]  ETA: 0:00:12

Optimising neural net:  91%[======================>  ]  ETA: 0:00:06

Optimising neural net: 100%[=========================] Time: 0:01:07


Machine trained 1 time; caches data
  model: NeuralNetworkRegressor(builder = MyNetworkBuilder(n1 = 16, …), …)
  args: 
    1:	Source @496 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @606 ⏎ `AbstractVector{Continuous}`


### Ensemble

In [27]:
# Compute predictions of all models, then stack them
MLmodels = [tree, forest_tm, xgb, nn]
M = zeros(length(y), 7)

M[:, 1] = predict(lm , df[!, X_cols])
M[:, 2] = predict(lasso_tm , X_flex)
M[:, 3] = predict(ridge_tm , X_flex)

for i in 1:length(MLmodels)
    M[:, i+3] = predict(MLmodels[i], X)
end
M[1, :]

7-element Vector{Float64}:
 30111.51965365291
 23977.178121897876
 22678.180822025963
  3674.0157498824638
 18423.78071031746
 11684.2265625
 13513.435143461189

In [28]:
using IJulia, Conda
Conda.add("scipy");

Solving environment: ...working... done

# All requested packages already installed.



In [35]:
# Compute stacking weights with Python's Scipy library
using PyCall

f(w) = (y .- M*w)' * (y .- M*w)

n_models = size(M)[2]
x0 = ones(n_models) * (1/n_models)

py"""
import numpy as np
import scipy.optimize as opt
cons = {'type':'eq', 'fun': lambda w: np.sum(np.abs(w))-1}
cons1 = ({'type':'eq', 'fun': lambda w: np.sum(np.abs(w))-1},
        {'type':'ineq', 'fun': lambda w: np.array(w)})


res = opt.minimize($f, $x0, constraints=cons)
"""
py"res"

Dict{Any, Any} with 9 entries:
  "fun"     => 1.17902e13
  "nit"     => 88
  "nfev"    => 950
  "status"  => 0
  "message" => "Optimization terminated successfully"
  "x"       => [2.57149e-8, 1.94315e-7, 2.53382e-8, 9.43755e-7, 0.999999, 1.166…
  "jac"     => [-5.129e11, -1.59061e12, -1.58479e12, -1.78138e12, -4.70491e12, …
  "success" => true
  "njev"    => 88

In [36]:
w = py"res.x"

7-element Vector{Float64}:
  2.571489836518568e-8
  1.9431493328132598e-7
  2.5338206435892112e-8
  9.437551277377716e-7
  0.9999987959331621
  1.1662625245493315e-7
 -5.0252783621024546e-8

## Mean-square forecast error of all models

In [37]:
X_test, y_test = df_test[!, X_cols], df_test[!, :net_tfa]

# Do data transformations on test set
X_test_f = copy(X_test)
nl_transform(X_test_f)
X_test_f = MLJ.transform(transformer, X_test_f)

X_test = machine(ContinuousEncoder(), X_test) |> fit! |> MLJ.transform;  # Onehot-encode categorical variables

In [38]:
# Compute predictions of all models, then stack them
MLmodels = [tree, forest_tm, xgb, nn]
G = zeros(length(y_test), 8)

G[:, 1] = predict(lm , df_test[!, X_cols])
G[:, 2] = predict(lasso_tm , X_test_f)
G[:, 3] = predict(ridge_tm , X_test_f)

for i in 1:length(MLmodels)
    G[:, i+3] = predict(MLmodels[i], X_test)
end


G[:, end] = w' .* G[:, 1:end-1] |> x -> sum(x, dims=2)  # Ensemble predictions

G[1, :]

8-element Vector{Float64}:
 -10875.710018469425
  -3893.503831663231
  -4606.913913835611
   3674.0157498824638
  -3164.0013174603173
  -2028.54833984375
   3623.919507091737
  -3163.995612078737

In [39]:
msfe = (G .- y_test).^2  # mean square forecast error
msfe_mean = mean(msfe, dims=1)
msfe_std = std(msfe, dims=1) / sqrt(length(y_test))

res = DataFrame(model=["LR", "Lasso", "Ridge", "Tree", "RF", "XGB", "NN", "Ensemble"],
          error=msfe_mean[:],
          std=msfe_std[:])
          
sort!(res, :error)

,model,error,std
,String,Float64,Float64
1,Ridge,2.85926e9,7.41807e8
2,Lasso,2.8771e9,7.45527e8
3,Ensemble,3.02558e9,7.79253e8
4,RF,3.02558e9,7.79253e8
5,XGB,3.1137e9,7.87585e8
6,LR,3.22963e9,8.07457e8
7,Tree,3.23919e9,7.91188e8
8,NN,3.86803e9,9.36655e8
